<a href="https://colab.research.google.com/github/balawillgetyou/dy/blob/master/FastAiPyTorchULMFiTTextClassifierWhoseLine_wStopWordsRemoval_20200630.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
#!pip install fastai

In [2]:
# import libraries
import codecs, os, numpy as np, pandas as pd
pd.set_option('display.max_colwidth', None) #to prevent cell display truncation

import re#for text preprocessing (checking for non words)

#metrics calculation
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score

#fastai
import fastai
from fastai import *
from fastai.text import * 
from functools import partial

#stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
temp = open("/content/gdrive/My Drive/Colab_Notebooks_Bala/WhoseLineIsItAnywayTRAIN.csv", 'r', encoding='latin-1')
WhoseLineData = pd.read_csv(temp)
print('File shape', WhoseLineData.shape, '\n')
print(WhoseLineData.columns, '\n')

File shape (18977, 2) 

Index(['text', 'author'], dtype='object') 



In [5]:
#Ran out of RAM for predictions. Hence dropping out records from some of the more frequent classes. 
WhoseLineData.author.value_counts()

0    3859
5    3517
4    3184
2    2621
3    1314
9    1142
7    1054
8     910
6     755
1     621
Name: author, dtype: int64

In [6]:
df_low=WhoseLineData[WhoseLineData.author.isin([1,6,8,7,9,3])]
df_high=WhoseLineData[~WhoseLineData.author.isin([1,6,8,7,9,3])]
g = df_high.groupby('author')
df_high_trimmed = pd.DataFrame(g.apply(lambda x: x.sample(2000)))
df = pd.concat([df_low,df_high_trimmed])
df = df.reset_index(drop=True)
df = pd.DataFrame({'label':df.author, 'text':df.text})
df.label.value_counts()

5    2000
4    2000
2    2000
0    2000
3    1314
9    1142
7    1054
8     910
6     755
1     621
Name: label, dtype: int64

In [7]:
#Data Preprocessing
#knock out the many white spaces & non words 
df['text'].replace(r'\s+', ' ', regex=True, inplace=True)
df['text'] = df['text'].str.lower().str.replace("[^a-zA-Z]", " ")

#unique word count
uniqueWords = set()
df['text'].str.split().apply(uniqueWords.update)
print('# of unique words',len(uniqueWords))
print('Sample of unique words',list(uniqueWords)[1070:1100])

#number of authors/ classes for the classifier
print("Authors in dataset", sorted(df['label'].unique()))

#to get an idea for the typical length of each text
df['textLength'] = df['text'].str.split().str.len()

#size of writings/ document
display(df.describe())

# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

# of unique words 60039
Sample of unique words ['later', 'bodkin', 'syllabus', 'tulle', 'impolitic', 'larch', 'niente', 'ur', 'talon', 'dialect', 'peski', 'restore', 'morton', 'iswar', 'promiseth', 'burlingame', 'dedicated', 'intrusion', 'indorse', 'fortified', 'fly', 'wakens', 'aisle', 'reindeer', 'pommeled', 'scowled', 'acumenus', 'malwa', 'woolloomoolloo', 'tagged']
Authors in dataset [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


,label,textLength
count,13796.000000,13796.000000
mean,4.061250,419.618730
std,2.751878,330.944833
min,0.000000,24.000000
25%,2.000000,258.000000
50%,4.000000,353.000000
75%,6.000000,490.000000
max,9.000000,5908.000000


In [100]:
# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.15, random_state = 35)
df_trn.shape, df_val.shape

((11726, 2), (2070, 2))

In [101]:
#data preparation for the language and classifier models

# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [102]:
#language model fine tuning on data from the target task, on which classification is to be performed
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)

In [103]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)#initial short increase in the learning rate to help the model to quickly converge 
#to a suitable region of the parameter space for the target task, followed by a long decay period which allows 
#for the further refining of the parameters

epoch,train_loss,valid_loss,accuracy,time
0,7.712512,7.457413,0.045315,04:59


In [104]:
learn.save_encoder('ft_enc_whoseLine_small')

In [105]:
#classifier training, using the fine tuned language model from above
#LSTM architecture selected. Transformers are also available in FastAi
#drop_mult controls layers wise drop outs. Setting =1 leaves them at their defaults
learn = text_classifier_learner(data_clas, drop_mult=0.7, arch=AWD_LSTM)
learn.load_encoder('ft_enc_whoseLine_small')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.071815,0.698460,0.768116,02:41


In [106]:
learn.save_encoder('txt_classifier_whoseLine_small')

In [108]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,252,0,12,0,42,44,0,8,5,13
1,0,68,0,0,0,0,0,9,0,1
2,11,1,254,2,1,30,0,7,15,4
3,1,0,4,189,5,7,0,1,1,4
4,19,0,0,0,232,9,0,3,0,4
5,2,1,13,1,1,149,0,1,9,5
6,5,1,1,0,4,2,103,7,2,12
7,2,22,1,0,3,13,5,113,0,2
8,0,0,14,0,0,17,0,1,103,0


In [109]:
precision, recall, fscore, support = score(targets, predictions, labels = [0,1,2,3,4,5,6,7,8,9])
results1 = pd.concat((pd.Series([0,1,2,3,4,5,6,7,8,9]), pd.Series(precision), pd.Series(recall), pd.Series(fscore), pd.Series(support)), axis=1, join = 'outer')
results1.columns=('label','precision', 'recall', 'fscore', 'support')
print('Results','\n','*'*50,'\n',round(results1,2))

Results 
 ************************************************** 
    label  precision  recall  fscore  support
0      0       0.67    0.84    0.75      300
1      1       0.87    0.73    0.80       93
2      2       0.78    0.85    0.81      300
3      3       0.89    0.96    0.92      197
4      4       0.87    0.77    0.82      300
5      5       0.82    0.50    0.62      300
6      6       0.75    0.91    0.82      113
7      7       0.70    0.72    0.71      158
8      8       0.76    0.75    0.76      137
9      9       0.64    0.74    0.69      172
